### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-08-14


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-08-14
Yesterday: 2025-08-13


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-08-13'
ORDER BY a.name



(0, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-08-13' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

RecursionError: maximum recursion depth exceeded

In [ ]:
df.shape

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,AIMIRT,2025-08-05,2025-08-08,9.85,9.90,"1,178,629",9.95,9.75,0.51%,B
1,DIF,2025-08-04,2025-08-08,8.20,8.45,"51,108,113",8.45,8.15,3.05%,B
2,GVREIT,2025-08-08,2025-08-08,6.65,6.65,"289,200",6.70,6.55,0.00%,B
3,JMT,2025-08-07,2025-08-08,12.10,12.00,"58,443,521",12.60,11.70,-0.83%,B
4,PTT,2025-08-06,2025-08-08,32.25,32.00,"154,034,467",32.50,31.75,-0.78%,B
5,SENA,2025-08-08,2025-08-08,1.78,1.78,"118,415",1.80,1.77,0.00%,B
6,WHAIR,2025-08-08,2025-08-08,5.50,5.50,"566,102",5.55,5.45,0.00%,B
7,WHART,2025-08-07,2025-08-08,9.30,9.30,"1,899,142",9.50,9.30,0.00%,B
8,IVL,2025-08-07,2025-08-08,23.40,22.80,"40,207,979",23.90,22.50,-2.56%,S
9,MCS,2025-08-04,2025-08-08,8.35,8.65,"2,033,365",8.70,8.30,3.59%,S


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(12, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(12, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,AIMIRT,2025-08-05,2025-08-08,9.85,9.90,1178629,9.95,9.75,0.51,B,9.90,0.00,142469,2025-08-13
1,DIF,2025-08-04,2025-08-08,8.20,8.45,51108113,8.45,8.15,3.05,B,8.45,0.00,20414690,2025-08-13
2,GVREIT,2025-08-08,2025-08-08,6.65,6.65,289200,6.70,6.55,0.00,B,6.80,0.15,622679,2025-08-13
3,JMT,2025-08-07,2025-08-08,12.10,12.00,58443521,12.60,11.70,-0.83,B,12.00,0.00,24463242,2025-08-13
4,PTT,2025-08-06,2025-08-08,32.25,32.00,154034467,32.50,31.75,-0.78,B,32.50,0.50,67251387,2025-08-13
5,SENA,2025-08-08,2025-08-08,1.78,1.78,118415,1.80,1.77,0.00,B,1.78,0.00,504906,2025-08-13
6,WHAIR,2025-08-08,2025-08-08,5.50,5.50,566102,5.55,5.45,0.00,B,5.45,-0.05,962035,2025-08-13
7,WHART,2025-08-07,2025-08-08,9.30,9.30,1899142,9.50,9.30,0.00,B,9.50,0.20,1510264,2025-08-13
8,IVL,2025-08-07,2025-08-08,23.40,22.80,40207979,23.90,22.50,-2.56,S,23.00,0.20,14646977,2025-08-13
9,MCS,2025-08-04,2025-08-08,8.35,8.65,2033365,8.70,8.30,3.59,S,8.75,0.10,1123225,2025-08-13


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AIMIRT   2025-08-05 2025-08-13   9.85   9.90   0.51%   1,178,629   9.95   9.75 B Update
DIF      2025-08-04 2025-08-13   8.20   8.45   3.05%  51,108,113   8.45   8.15 B Update
GVREIT   2025-08-08 2025-08-13   6.65   6.80   2.26%     289,200   6.80   6.55 B Update
IVL      2025-08-13 2025-08-13  22.80  23.00   0.88%  40,207,979  23.90  22.50 S Insert
JMT      2025-08-07 2025-08-13  12.10  12.00  -0.83%  58,443,521  12.60  11.70 B Update
MCS      2025-08-04 2025-08-13   8.35   8.75   4.79%   2,033,365   8.75   8.30 S Update
NER      2025-08-05 2025-08-13   4.46   4.58   2.69%  28,929,471   4.60   4.40 S Update
PTG      2025-08-13 2025-08-13   6.75   7.05   4.44%   8,960,653   7.05   6.65 S Insert
PTT      2025-08-13 2025-08-13  32.00  32.50   1.56% 154,034,467  32.50  31.75 B Insert
SENA     2025-08-08 2025-08-13  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [55]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-08-13' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [57]:
df = pd.read_sql(sql, conlite)
df.shape

(0, 10)

In [59]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

RecursionError: maximum recursion depth exceeded

In [ ]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [ ]:
df_sort.query('percent <= -5.00').style.format(format_dict)

In [ ]:
df_sort.query('percent >= 5.00').style.format(format_dict)

In [ ]:
conlite.close()

In [ ]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)